# Dash

Dash from plotly is a library for rapidly building data apps, including interactive components and visualisations.

Documentation:
* [Dash](https://dash.plotly.com/)
* Plotting is done with [plotly](https://plotly.com/python/)
* [Dash Bootstrap Components](https://dash-bootstrap-components.opensource.faculty.ai/) to style and structure your app

The documentation is pretty good, so here we will just go through the basics, look at interesting features and show how you can structure a complete app.

## Layout

With the layout you define what the app looks like.

A simple Dash app uses simple [HTML Components](https://dash.plotly.com/dash-html-components) and [Dash Core Components](https://dash.plotly.com/dash-core-components) for interactive components.

**Setup**

In [ ]:
# Import packages and data
import datetime
from pathlib import Path

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc

from src.load_profiles import get_load_profile

In [ ]:
# Get Lastprofil data
lastprofile_file = (
    Path(".") / "external" / "Lastprofile" / "representative_profiles_vdew.xls"
)
h0_2023 = get_load_profile(
    lastprofile_file,
    from_=datetime.date(2023, 1, 1),
    to=datetime.date(2023, 12, 31),
    type="H0",
)

In [ ]:
# Make plot with plotly
h0fig = px.line(h0_2023)
h0fig.update_xaxes(title_text="Date", range=["2023-03-01", "2023-03-14"])
h0fig.update_yaxes(title_text="Power [W]");

### Small Dash App

In [ ]:
# Initialize the app
app = Dash(__name__)

app.layout = html.Div(
    [  # Standard html container
        html.H2(children="H0 Load Profile"),  # Level 2 Heading
        dcc.Graph(figure=h0fig),  # Graph component from Dash Core Components
    ]
)

app.run(debug=True, port=8001)

### Further Layout Components

* For organizing the app we use [Dash Bootstrap Components](https://dash-bootstrap-components.opensource.faculty.ai/), can also directly use components from there
* dcc has also many input components
* [DataTable](https://dash.plotly.com/datatable) is very useful for tabular data, with also interactive possibilities.
* Maps can be created by using [plotly Mapbox](https://plotly.com/python/maps/) plots.

In [ ]:
# Layout for further example with callbacks. Has to use component ids for interactive components.

# Dropdown for Profile Type, date range selector, plot.
# Use dbc.Row and Column

**Map of Vienna**

For Access Token follow  https://plotly.com/python/mapbox-layers/

In [ ]:
mapbox_access_token = "pk.eyJ1IjoiY2hhbmNoZWVrZWFuIiwiYSI6ImNqdjgzYmYzNjBmeDQzem43MzIwcnI1djMifQ.igdgIdtTUOVIAXO7WA2ZBw"
px.set_mapbox_access_token(mapbox_access_token)

In [ ]:
data = pd.DataFrame(
    {
        "station_id": ['hier', 'da', 'dort', 'links', 'rechts', 'weg', 'null'],
        "lat": [48.227, 48.235, 48.234, 48.229, 48.230, 48.227, 48.224],
        "long": [16.502, 16.474, 16.474, 16.475, 16.495, 16.491, 16.498],
        "cluster": ["c1", "c2", "c3", "c4", "c3", "c2", "c3"],
        "annual": [0.2, 0.3, 0.2, 0.4, 0.2, 0.3, 0.2],
    }
)

In [ ]:
map_vie = px.scatter_mapbox(
    data,
    lat="lat",
    lon="long",
    size="annual",
    color="cluster",
    custom_data=[data.station_id],
    hover_name=data.station_id,
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=10,
    zoom=9,
    category_orders={"cluster": ["c1", "c2", "c3", "c4"]},
)

map_vie.update_layout(
    autosize=True,
    showlegend=True,
    hovermode="closest",
    dragmode="select",
    clickmode="event+select",
    margin=dict(l=20, r=0, t=20, b=20),
    font=dict(size=10),
    mapbox=dict(accesstoken=mapbox_access_token, bearing=0, style="dark", pitch=0),
);

In [ ]:
app = Dash(__name__)


app.layout = dbc.Card(
    [
        dbc.CardHeader("Trafos Clustered", className="card_header"),
        dcc.Graph(
            figure=map_vie, config=dict(scrollZoom=True), style=dict(height="600px")
        ),
    ],
    color="dark",
    outline=True,
    className="mb-2",
)

app.run(debug=True, port=8002)

## Callbacks

simple example making the above responsive. 

Further topics:
* Possibility of using State.
* PreventUpdate and prevent_initial_call
* allow_duplicate

## App structure and general remarks

For deploying the app, I suggest to break apart the main parts of the app into at least three files:

```
- app.py
- layout.py
- callbacks.py
```

It is best to not mix the programming logic with the app, so load the functions in *callbacks.py*,
but define them in another file, folder or package.

I added an example Dash App in the *app/* folder. It can be run directly by 

```bash
python app/app.py
```

for local development. If you want to run a more stable version, that can also be accessed from the network, use
a WSGI like waitress or gunicorn. This is done in *wsgi.py* but can also be done directly from the command line:

```bash
waitress-serve app:server --host=0.0.0.0 --port 8050
```